In [1]:
import geostatspy.GSLIB as GSLIB          # GSLIB utilies, visualization and wrapper
import geostatspy.geostats as geostats    # GSLIB methods convert to Python

import numpy as np                        # ndarrys for gridded data
import pandas as pd                       # DataFrames for tabular data
import os                                 # set working directory, run executables
import matplotlib.pyplot as plt           # for plotting
from scipy import stats                   # summary statistics
import math                               # trig etc.
import random

import itertools
from itertools import combinations

In [2]:
# Setting directory
os.chdir("/Users/wnguyen/Dropbox/CR/mcfigs") # set the working directory

In [3]:
# Importing data
data_url = 'https://raw.githubusercontent.com/wdnguyen/CR/master/CostaRica_Chemistry_20200518.csv'

# Read data from url as pandas dataframe
df = pd.read_csv(data_url)

df['SamplingDate'] = pd.to_datetime(df['SamplingDate'])
df['Chemetrics_Acidified_Date'] = pd.to_datetime(df['Chemetrics_Acidified_Date'])

# Filtering endmembers
df_rain = df[df['Site'] == "Rain"].reset_index(drop=True) # Endmember 1 = Rain
df_soil = df[df['Site'] == "Soil"].reset_index(drop=True) # Endmember 2 = Soil
df_spring = df[df['Site'] == "Spring"].reset_index(drop=True) # Endmember 3a = Spring
df_us = df[(df['Site'] == "Upstream") & (df['SamplingDate'] < '2019')].reset_index(drop=True) # Endmember 3b = Upstream, not including 2019

# Filtering mixing batches
df_ds = df[(df['Site'] == "Downstream") & (df['SamplingDate'] < '2019')] # Mixing batch a, not including 2019
df_ds = df_ds[~df_ds['ID'].str.contains("Howler")] # Only want the DS samples
df_ds = df_ds[~df_ds['ID'].str.contains("DS9Bot")].reset_index(drop=True) # Filtering out DS9Bot (duplicate)
df_stream = df[(df['Source'] == "Stream") & (df['SamplingDate'] < '2019')].reset_index() # Mixing batch b, not including 2019

# Summary statistics:
df_summary = df.groupby("Site").describe()

# For this analysis, I'm dropping the US samples that are missing 
# from the DS samples (because the ISCO was not installed correctly):
# Samples 4 - 8, 10
df_us = df_us.drop(df_us.index[3:8]).reset_index(drop=True)
# df_us = df_us.drop(df_us.index[4]).reset_index(drop=True) # Dropping sample 10
# df_ds = df_ds.drop(df_ds.index[4]).reset_index(drop=True) # Dropping sample 10

# Sample 10 exists for O18 and D, do this one later

In [ ]:
#SO4 and Cl
# Endmembers: up, rain, soil
# Mixing: down
# up endmember is moving with mixing batch down # assume no error for now
# applying normal distribution to rain and soil endmembers

# 1 = SO4
# 2 = Cl

rain_1_mean = df_summary['SO4']['mean']['Rain']; rain_1_stdev = df_summary['SO4']['std']['Rain'] # Gaussian mean and standard deviation for rain endmember
rain_2_mean = df_summary['Cl']['mean']['Rain']; rain_2_stdev = df_summary['Cl']['std']['Rain'] # Gaussian mean and standard deviation for rain endmember
soil_1_mean = df_summary['SO4']['mean']['Soil']; soil_1_stdev = df_summary['SO4']['std']['Soil'] # Gaussian mean and standard deviation for soil endmember
soil_2_mean = df_summary['Cl']['std']['Soil']; soil_2_stdev = df_summary['Cl']['std']['Soil'] # Gaussian mean and standard deviation for soil endmember
L = 1000 # Number of MCS realizations

# Stdev of rain Cl will be negative sometimes...make sure to correct for that later

In [ ]:
# Apply normal distribution to draw L realizations for each variable and storing them in ndarrays
rain_1 = np.random.normal(rain_1_mean, rain_1_stdev, size = L)
rain_2 = np.random.normal(rain_2_mean, rain_2_stdev, size = L)
soil_1 = np.random.normal(soil_1_mean, soil_1_stdev, size = L)
soil_2 = np.random.normal(soil_2_mean, soil_2_stdev, size = L)

In [ ]:
# Plotting distribution of the realizations fr each variable
rain_1_min = rain_1_mean - rain_1_stdev; rain_1_max = rain_1_mean + rain_1_stdev            # average porosity min and max
rain_2_min = rain_2_mean - rain_2_stdev; rain_2_max = rain_2_mean + rain_2_stdev
soil_1_min = soil_1_mean - soil_1_stdev; soil_1_max = soil_1_mean + soil_1_stdev
soil_2_min = soil_2_mean - soil_2_stdev; soil_2_max = soil_2_mean + soil_2_stdev 

plt.subplot(141)
GSLIB.hist_st(rain_1,rain_1_min,rain_1_max,log=False,cumul=False,bins=50,weights=None,xlabel="Rain SO4 (mgL-1)",title="Average Rain SO4 Realizations")
plt.ylim(0.0,30)

plt.subplot(142)
GSLIB.hist_st(rain_2,rain_2_min,rain_2_max,log=False,cumul=False,bins=50,weights=None,xlabel="Rain Cl (mgL-1)",title="Average Rain Cl Realizations")
plt.ylim(0.0,30)

plt.subplot(143)
GSLIB.hist_st(soil_1,soil_1_min,soil_1_max,log=False,cumul=False,bins=50,weights=None,xlabel="Soil SO4 (mgL-1)",title="Average Soil SO4 Realizations")
plt.ylim(0.0,30)

plt.subplot(144)
GSLIB.hist_st(soil_2,soil_2_min,soil_2_max,log=False,cumul=False,bins=50,weights=None,xlabel="Soil Cl (mgL-1)",title="Average Soil Cl Realizations")
plt.ylim(0.0,30)

plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.2, wspace=0.2, hspace=0.2)
plt.show()

In [ ]:
up_1 = np.repeat(0.9284, L)
up_2 = np.repeat(2.2664, L)
down_1 = np.repeat(1.1921, L)
down_2 = np.repeat(2.2138, L)
ones = np.repeat(1, L)

In [ ]:
# Linear equations:
# up_f = (down_1 - soil_1 - rain_f(rain_1 - soil_1))/(up_1 - soil_1)
# soil_f = (down_2 - up_2 - rain_f(rain_2 - up_2))/(soil_2 - up_1)
# rain_f = 1 - up_f - soil_f

In [ ]:
results = []
for x in range(0, L):
    a = np.array([[up_1[x], soil_1[x], rain_1[x]], [up_2[x], soil_2[x], rain_2[x]], [ones[x], ones[x], ones[x]]])
    b = np.array([down_1[x], down_2[x], ones[x]])
    results.append(np.linalg.solve(a,b))

# np.hstack(results)

In [ ]:
up_f = []
soil_f = []
rain_f = []
for x in range(0,L):
    up_f.append(results[x][0])
    soil_f.append(results[x][1])
    rain_f.append(results[x][2])

In [ ]:
df_ds

In [8]:
# Nested loop: # more info: https://docs.python.org/3/library/itertools.html#itertools.combinations
# https://stackoverflow.com/questions/942543/operation-on-every-pair-of-element-in-a-list/37907649

var = ['SO4','Cl','O18','D'] # variables of interest
L = 1000 # Number of MCS realizations
ones = np.repeat(1, L) # numpy array of L ones
save_results_to = '/Users/wnguyen/Dropbox/CR/mcfigs'
# list(itertools.combinations(var, 2)) # generates list of combinations

for pair in itertools.combinations(var, 2):
    # 1) Deal with endmember pairs: Rain and Soil
    rain_1_mean = df_summary[pair[0]]['mean']['Rain']
    rain_1_stdev = df_summary[pair[0]]['std']['Rain']
    rain_2_mean = df_summary[pair[1]]['mean']['Rain']
    rain_2_stdev = df_summary[pair[1]]['std']['Rain']
    soil_1_mean = df_summary[pair[0]]['mean']['Soil']
    soil_1_stdev = df_summary[pair[0]]['std']['Soil']
    soil_2_mean = df_summary[pair[1]]['mean']['Soil']
    soil_2_stdev = df_summary[pair[1]]['std']['Soil']
    # 2) Apply normal distribution to draw L realizations for each variable and storing them in ndarrays
    rain_1 = np.random.normal(rain_1_mean, rain_1_stdev, size = L)
    rain_2 = np.random.normal(rain_2_mean, rain_2_stdev, size = L)
    soil_1 = np.random.normal(soil_1_mean, soil_1_stdev, size = L)
    soil_2 = np.random.normal(soil_2_mean, soil_2_stdev, size = L)
    # 3) Loop through each sample
    for x in range(0, len(df_ds)): # Should be DS1 - DS26, minus the ones taken out earlier
        sample = df_ds['ID'][x] # saving sample name for graphs (i.e. 'DS1')
        up_1 = np.repeat((df_us[pair[0]][x]), L)
        up_2 = np.repeat((df_us[pair[1]][x]), L)
        down_1 = np.repeat((df_ds[pair[0]][x]), L)
        down_2 = np.repeat((df_ds[pair[1]][x]), L)
        results = []
        for i in range(0, L):
            a = np.array([[up_1[i], soil_1[i], rain_1[i]], [up_2[i], soil_2[i], rain_2[i]], [ones[i], ones[i], ones[i]]])
            b = np.array([down_1[i], down_2[i], ones[i]])
            results.append(np.linalg.solve(a,b))
        up_f = []
        soil_f = []
        rain_f = []
        for k in range(0,L):
            up_f.append(results[k][0])
            soil_f.append(results[k][1])
            rain_f.append(results[k][2])
        # Plots
        plt.subplot(311)
        GSLIB.hist_st(up_f,-1,2,log=False,cumul=False,bins=50,weights=None,xlabel="Up Fraction",title="Up Fraction: {0} with {1} and {2}".format(sample, pair[0], pair[1]))
        plt.ylim(0.0,500)

        plt.subplot(312)
        GSLIB.hist_st(soil_f,-1,2,log=False,cumul=False,bins=50,weights=None,xlabel="Soil Fraction",title="Soil Fraction: {0} with {1} and {2}".format(sample, pair[0], pair[1]))
        plt.ylim(0.0,500)

        plt.subplot(313)
        GSLIB.hist_st(rain_f,-1,2,log=False,cumul=False,bins=50,weights=None,xlabel="Rain Fraction",title="Rain Fraction: {0} with {1} and {2}".format(sample, pair[0], pair[1]))
        plt.ylim(0.0,500)

        plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=1.2, wspace=0.2, hspace=0.8)
        
        plt.savefig("Graph" + str(sample) + str(pair[0]) + str(pair[1]) + ".png", bbox_inches = 'tight')
        plt.clf()

<Figure size 432x288 with 0 Axes>

In [ ]:
plt.subplot(311)
GSLIB.hist_st(up_f,-1,2,log=False,cumul=False,bins=50,weights=None,xlabel="Up Fraction",title="Up Fraction: {0} with {1} and {2}".format(sample, pair[0], pair[1]))
plt.ylim(0.0,500)

plt.subplot(312)
GSLIB.hist_st(soil_f,-1,2,log=False,cumul=False,bins=50,weights=None,xlabel="Soil Fraction",title="Soil Fraction: {0}".format(sample))
plt.ylim(0.0,500)

plt.subplot(313)
GSLIB.hist_st(rain_f,-1,2,log=False,cumul=False,bins=50,weights=None,xlabel="Rain Fraction",title="Rain Fraction: {0}".format(sample))
plt.ylim(0.0,500)

plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=1.2, wspace=0.2, hspace=0.8)
plt.show()